In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as cdf
import xarray as xr

In [2]:
with xr.open_dataset('/Users/taiwanesepapaya/codeplay/Wood Harvesting/Masks & grids/paris_countrymask_0.2x0.1deg_2D.nc') as ds:
    paris_masks = ds

df = pd.read_csv('/Users/taiwanesepapaya/codeplay/Wood Harvesting/FAO Wood Production Maps/AllWood_2006-2022.csv')

In [3]:
df.loc[(df['Element']=='Export Quantity'), 'Value']*= -1

In [4]:
nt_df = df[['Item', 'Area', 'Year', 'Value']].groupby(['Item', 'Area', 'Year']).sum().reset_index()

In [5]:
nt_df.head(3)

,Item,Area,Year,Value
0,Paper and paperboard,Albania,2006,19725.0
1,Paper and paperboard,Albania,2007,16925.0
2,Paper and paperboard,Albania,2008,16925.0


Sawnwood decay patterns & C flux

In [6]:
#Sawnwood
# density_sawnwood = 0.458
# c_fraction_sawnwood = 0.5
# Dry weight of wood to volume ratio = 410kg/m3
# FAO Value is reported as volume (m3)

nt_finland_sw = nt_df[(nt_df['Area']== 'Finland')& (nt_df['Item'] == 'Sawnwood')]

# Convert volume to dry weight and get carbon fraction per weight
nt_finland_sw['Inflow_c'] = nt_finland_sw['Value']/410*0.5
print(nt_finland_sw['Inflow_c'])

1656    6191.948780
1657    7345.369512
1658    5313.962195
1659    4250.195122
1660    5197.785366
1661    5034.260976
1662    4202.369512
1663    4440.064634
1664    4646.719512
1665    3885.164634
1666    3968.241463
1667    3583.434146
1668    4582.780488
1669    3677.120732
1670    4022.173171
1671    4643.951220
1672    3695.469512
Name: Inflow_c, dtype: float64


/var/folders/p6/2mrc_c6s56jfx77j1t1ryrw40000gn/T/ipykernel_91369/1481199992.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nt_finland_sw['Inflow_c'] = nt_finland_sw['Value']/410*0.5


In [7]:
# Define function for sawnwood decay based on half-life of 35 years

#Input variables:
half_life = 35                                  # years
k = np.log(2)/half_life                         # decay constant for sawnwood
Inflow_t0 = nt_finland_sw['Inflow_c'].mean()    # Initial carbon inflow of sawnwood
c_t0 = Inflow_t0 / k                            # Initial carbon stock of sawnwood

def first_order_decay_sw(k=k , Inflow= Inflow_t0, c_i= c_t0): 
    return np.exp(-1*k)*c_i + ((1-np.exp(-1*k))/k)*Inflow

print(first_order_decay_sw(k=k , Inflow= Inflow_t0, c_i= c_t0))

first_order_decay_sw()

233702.6265960283


233702.6265960283

In [8]:
Inflow_t0 = nt_finland_sw['Inflow_c'].mean()    # Initial carbon inflow of sawnwood
c_t0 = Inflow_t0 / k     

c_t = c_t0
inflow_c_t = Inflow_t0

for time in nt_finland_sw['Year']:
    
    c_t = first_order_decay_sw(k=k,Inflow = inflow_c_t, c_i = c_t)
    inflow_c_t = nt_finland_sw[nt_finland_sw['Year']==time]["Inflow_c"].values

    print(c_t)

233702.6265960283
[235250.89885926]
[237910.88491378]
[238507.28601707]
[238038.68932285]
[238517.55026847]
[238825.10525156]
[238302.9211695]
[238026.33370024]
[237959.79194474]
[237140.49161332]
[236419.51685942]
[235331.6580352]
[235254.64717335]
[234282.39571825]
[233670.86765961]
[233686.99287577]


Wood Based Panels Decay Pattern & C flux

In [9]:
# Wood Based Panels (WBP)
density_wbp = 0.595
c_fraction_wbp = 0.454
# Dry weight of wood to volume ratio = 410kg/m3
# FAO Value is reported as volume (m3)

nt_finland_wbp = nt_df[(nt_df['Area']== 'Finland')& (nt_df['Item'] == 'Wood-based panels')]

# Convert volume to dry weight and get carbon fraction per weight
nt_finland_wbp['Inflow_c'] = nt_finland_wbp['Value']/410*c_fraction_wbp
print(nt_finland_wbp['Inflow_c'])

3100    874.209112
3101    944.658839
3102    883.837234
3103    673.407127
3104    786.393332
3105    755.452678
3106    758.816707
3107    713.924966
3108    652.530878
3109    606.854049
3110    682.697517
3111    692.352215
3112    729.532600
3113    689.280517
3114    714.900512
3115    685.583185
3116    671.910034
Name: Inflow_c, dtype: float64


/var/folders/p6/2mrc_c6s56jfx77j1t1ryrw40000gn/T/ipykernel_91369/684874457.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nt_finland_wbp['Inflow_c'] = nt_finland_wbp['Value']/410*c_fraction_wbp


In [10]:
# Define function for WBP decay based on half-life of 25 years

#Input variables:
half_life_wbp = 25                                      # years
k_wbp = np.log(2)/half_life_wbp                         # decay constant for sawnwood
Inflow_t0_wbp = nt_finland_wbp['Inflow_c'].mean()       # Initial carbon inflow of sawnwood
c_t0_wbp = Inflow_t0_wbp / k_wbp                        # Initial carbon stock of sawnwood

def first_order_decay_wbp(k=k_wbp , Inflow= Inflow_t0_wbp, c_i= c_t0_wbp): 
    return np.exp(-1*k)*c_i + ((1-np.exp(-1*k))/k)*Inflow

print(first_order_decay_wbp(k=k_wbp , Inflow= Inflow_t0_wbp, c_i= c_t0_wbp))

first_order_decay_wbp()

26554.79972888456


26554.79972888456

In [11]:
Inflow_t0_wbp = nt_finland_wbp['Inflow_c'].mean()    # Initial carbon inflow of sawnwood
c_t0_wbp = Inflow_t0_wbp / k_wbp    

c_t = c_t0_wbp
inflow_c_t = Inflow_t0_wbp

for time in nt_finland_wbp['Year']:
    
    c_t = first_order_decay_wbp(k=k_wbp,Inflow = inflow_c_t, c_i = c_t)
    inflow_c_t = nt_finland_wbp[nt_finland_wbp['Year']==time]["Inflow_c"].values

    print(c_t)

26554.79972888456
[26690.85856671]
[26892.67891856]
[27028.99430508]
[26954.04243895]
[26992.57439513]
[26999.53703101]
[27009.62709519]
[26975.16612434]
[26881.09669387]
[26744.55017116]
[26686.53922501]
[26639.6366747]
[26630.68636363]
[26582.28160712]
[26560.46856764]
[26510.33737421]


Paper & Paperboard decay pattern & C flux

In [15]:
# Paper & Paperboard (P&P)
# Note that paper is not in density 
relative_dry_mass_pp = 0.9
c_fraction_pp = 0.429

nt_finland_pp = nt_df[(nt_df['Area']== 'Finland')& (nt_df['Item'] == 'Paper and paperboard')]

nt_finland_pp['Inflow_c'] = nt_finland_pp['Value']*relative_dry_mass_pp*c_fraction_pp

nt_finland_pp['Inflow_c'].head(3)


/var/folders/p6/2mrc_c6s56jfx77j1t1ryrw40000gn/T/ipykernel_91369/301579933.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nt_finland_pp['Inflow_c'] = nt_finland_pp['Value']*relative_dry_mass_pp*c_fraction_pp


204    672642.9567
205    664027.9074
206    683874.2196
Name: Inflow_c, dtype: float64

In [16]:
# Define function for WBP decay based on half-life of 25 years

#Input variables:
half_life_pp = 2                                      # years
k_pp = np.log(2)/half_life_pp                         # decay constant for sawnwood
Inflow_t0_pp = nt_finland_pp['Inflow_c'].mean()       # Initial carbon inflow of sawnwood
c_t0_pp = Inflow_t0_pp / k_pp                         # Initial carbon stock of sawnwood

def first_order_decay_pp(k=k_pp , Inflow= Inflow_t0_pp, c_i= c_t0_pp): 
    return np.exp(-1*k)*c_i + ((1-np.exp(-1*k))/k)*Inflow

print(first_order_decay_pp(k=k_pp , Inflow= Inflow_t0_pp, c_i= c_t0_pp))

first_order_decay_pp()

1298421.0272917997


1298421.0272917997

In [17]:
c_t = c_t0_pp
inflow_c_t = Inflow_t0_pp

for time in nt_finland_pp['Year']:
    
    c_t = first_order_decay_pp(k=k_pp,Inflow = inflow_c_t, c_i = c_t)
    inflow_c_t = nt_finland_pp[nt_finland_pp['Year']==time]["Inflow_c"].values

    print(c_t)

1298421.0272917997
[1486580.40186286]
[1612348.49701224]
[1718052.31044165]
[1660435.27275803]
[1626277.90424084]
[1592144.86414354]
[1591075.45164617]
[1509394.88981967]
[1425118.1870449]
[1283868.32199151]
[1217822.05988848]
[1137514.60147929]
[1077875.50833114]
[1001076.63898767]
[930121.6587313]
[863838.628026]


Fuel Wood C flux = C pool because we assume instant expiration

In [20]:
nt_finland_fw = nt_df[(nt_df['Area']== 'Finland')& (nt_df['Item'] == 'Wood fuel')]
nt_finland_fw['Inflow_c'] = nt_finland_fw['Value']/410 * 0.5
print(nt_finland_fw['Inflow_c'])

2386     6652.809756
2387     6527.428049
2388     6958.939024
2389     8401.410976
2390     8339.385366
2391     8885.154878
2392     9393.860976
2393     9303.174390
2394     9427.231707
2395     9586.076829
2396     8566.391463
2397     9633.731707
2398     9335.978049
2399     9770.380488
2400    11005.728049
2401    10982.731707
2402    11508.680488
Name: Inflow_c, dtype: float64


/var/folders/p6/2mrc_c6s56jfx77j1t1ryrw40000gn/T/ipykernel_91369/1267668925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nt_finland_fw['Inflow_c'] = nt_finland_fw['Value']/410 * 0.5


C flux for 2022

In [21]:
# change in stock between 2022 and 2021, accounding for different decay patterns
# Units: tC/yr

delta_c_sw = (233686.99287577 - 233670.86765961)/1000
delta_c_wbp = (26510.33737421 - 26560.46856764)/1000
delta_c_pp = 863838.628026 - 930121.6587313
delta_c_fw = (11508.680488 - 10982.731707)/1000

delta_c_wood_products = delta_c_sw + delta_c_wbp + delta_c_pp + delta_c_fw
print(delta_c_wood_products)

-66282.53876249632


C flux from wood products in Finland for 2021

In [51]:
# change in stock between 2021 and 2020, accounding for different decay patterns
# Units: tC/yr

delta_c_sw = (233670.86765961 - 234282.39571825)/1000
delta_c_wbp = (26560.46856764 - 26582.28160712)/1000
delta_c_pp = 930121.6587313 - 1001076.63898767
delta_c_fw = (10982.731707 -   11005.728049)/1000

delta_c_wood_products_2021 = delta_c_sw + delta_c_wbp + delta_c_pp + delta_c_fw
print(delta_c_wood_products_2021)

-70955.63659381004


In [52]:
c_flux_wood_product_fin_2021_molC_m2_sec = (((delta_c_wood_products_2021 * 1e+9) / 12)/(338500 * 1e+6))/(365*24*60*60)
print(c_flux_wood_product_fin_2021_molC_m2_sec)

-5.539114910057386e-07


In [39]:
print(delta_c_fw/1e+9)

5.259487809999992e-10


In [33]:
# tC to PgC
delta_c_wood_products_PgC = delta_c_wood_products / 1e+9

print(delta_c_wood_products_PgC)

-6.628253876249631e-05


In [48]:
# Area of Finland 31,536,000 m2 or 338,500 km2

delta_c_wood_products_PgC_km2 = delta_c_wood_products_PgC / 338500
delta_c_wood_products_PgC_m2 = delta_c_wood_products_PgC / (338500 * 1e+6)


print(delta_c_wood_products_PgC_m2)

-1.9581252219348986e-16


In [49]:
delta_c_wood_products_molC_m2 = (delta_c_wood_products_PgC_m2 * 1e+18) / 12
print(delta_c_wood_products_molC_m2)

-16.317710182790822


In [50]:
delta_c_wood_products_molC_m2_sec = delta_c_wood_products_molC_m2 / (365*24*60*60)
print(delta_c_wood_products_molC_m2_sec)

-5.174311955476542e-07


In [35]:
PgCO2_km2_EU27_woodproduct = delta_c_wood_products_PgC_km2 * 4225127 * 3.664
print(PgCO2_km2_EU27_woodproduct)

-0.0030313472856134276
